In [1]:

%matplotlib inline
from itertools import chain

from cmdstanpy import CmdStanModel
import numpy as np
import pandas as pd
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()
import importlib


/home/josh/workspace/bayesianquilts/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_probability.python.bijectors as tfb

from bayesianquilts.models.logistic_regression_reparam import LogisticRegression2
from bayesianquilts.metrics.classification import classification_metrics
from bayesianquilts.sampler import psis, nppsis

2024-12-03 15:43:19.842713: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 15:43:19.846286: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 15:43:19.856192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733240599.872087 1336017 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733240599.876802 1336017 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 15:43:19.894933: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [5]:
with importlib.resources.path('bayesianquilts', 'data') as data_path:
    roach_path = data_path / "roachdata.csv"
    df = pd.read_csv(roach_path)

In [7]:


y_ = (df['y']>0).astype(int)
X_ = df.iloc[:, 2:]
X_scaled = (X_ - X_.mean())/X_.std()
X_scaled = X_scaled.fillna(0).to_numpy()
y_ = y_.to_numpy()
N = X_scaled.shape[0]
d = X_scaled.shape[1]

print((N, d))

(262, 4)


In [8]:
with importlib.resources.path('bayesianquilts', 'stanmodels') as data_path:
    model_path = data_path / "horseshoe_logistic.stan"
    sm = CmdStanModel(stan_file=model_path)

15:46:08 - cmdstanpy - INFO - compiling stan file /home/josh/workspace/bayesianquilts/bayesianquilts/stanmodels/horseshoe_logistic.stan to exe file /home/josh/workspace/bayesianquilts/bayesianquilts/stanmodels/horseshoe_logistic
15:46:24 - cmdstanpy - INFO - compiled model executable: /home/josh/workspace/bayesianquilts/bayesianquilts/stanmodels/horseshoe_logistic


In [ ]:
guessnumrelevcov = 4 # 20.
slab_scale = 2.5/100
scale_icept = 5.0
nu_global = 1
nu_local = 1
slab_df = 1
scale_global = guessnumrelevcov / ((d - guessnumrelevcov) * np.sqrt(N))/10000

control = {"adapt_delta": 0.9995, "max_treedepth": 15}

roach_data = {
    "N": N,
    "d": d,
    "slab_df": slab_df,
    "slab_scale": slab_scale,
    "scale_icept": scale_icept,
    "nu_global": 1,
    "nu_local": 1,
    "scale_global": np.abs(scale_global),
    "y": y_.astype(int)[:].tolist(),
    "x": X_scaled.tolist(),
}

import json
with open(f"{importlib.resources.path('bayesianquilts', 'data')}/roach_data.json", 'w') as f:
    json.dump(roach_data, f)

In [ ]:
fit = sm.sample(
    data=f"{importlib.resources.path('bayesianquilts', 'data')}/roach_data.json",
    iter_warmup=20000,
    iter_sampling=2000,
    thin=2,
    adapt_delta=0.9995,
    max_treedepth=15,
)

15:48:09 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status





chain 1 |          | 00:00 Iteration:     1 / 22000 [  0%]  (Warmup)







chain 1 |▏         | 00:00 Iteration:   100 / 22000 [  0%]  (Warmup)




chain 1 |▏         | 00:01 Iteration:   200 / 22000 [  0%]  (Warmup)




chain 1 |▏         | 00:01 Iteration:   300 / 22000 [  1%]  (Warmup)


chain 1 |▎         | 00:01 Iteration:   400 / 22000 [  1%]  (Warmup)


chain 1 |▎         | 00:01 Iteration:   500 / 22000 [  2%]  (Warmup)



chain 1 |▎         | 00:01 Iteration:   600 / 22000 [  2%]  (Warmup)
chain 1 |▍         | 00:01 Iteration:   700 / 22000 [  3%]  (Warmup)



chain 1 |▍         | 00:02 Iteration:   800 / 22000 [  3%]  (Warmup)
chain 1 |▍         | 00:02 Iteration:   900 / 22000 [  4%]  (Warmup)


chain 1 |▌         | 00:02 Iteration:  1000 / 22000 [  4%]  (Warmup)
chain 1 |▌         | 00:02 Iteration:  1100 / 22000 [  5%]  (Warmup)


chain 1 |▋         | 00:02 Iteration:  1200 / 220